In [1]:
import pandas as pd

In [2]:
RAW_FILENAME = "lol_C_M_GM_processed_normalized_D04-19-2024T12_33_31.csv"

lol_df = pd.read_csv(RAW_FILENAME)

In [4]:
lol_df["tier"].value_counts()

tier
2    7187
1     700
0     300
Name: count, dtype: int64

In [9]:
duplicates = lol_df.duplicated(subset=['username'], keep=False)
duplicate_rows = lol_df[duplicates].sort_values(by="username")
duplicate_rows.head(10)

,username,tagline,tier,level,total_wins,total_losses,past_n_wins,past_n_losses,damage_dealt_to_champions,damage_taken,...,deaths,assists,kill_participation,creep_score,players_played_with,adc,jungle,mid,support,top
3265,888,NA2,2,511,38,34,11,9,19012.95,23117.55,...,5.50,5.45,0.472795,189.40,"['xSantaBearx#NA1', 'The Krow#NA1', 'best lol ...",0,0,1,0,0
4393,888,1995,2,217,244,237,10,10,19249.25,18381.35,...,4.20,4.95,0.351085,191.75,"['UW Scrimm#1v9', 'Cute Timmo#NA1', 'Black Lot...",0,0,0,0,1
3300,ADC,ZYH,1,53,83,47,11,9,19322.15,17944.65,...,5.50,7.05,0.488722,200.60,"['Love Ego#1tap', 'Crucile#NA1', 'ttv ssyeu#ss...",1,0,0,0,0
7382,ADC,MUNCH,2,670,340,353,8,12,20643.00,20190.95,...,6.85,5.20,0.515275,174.70,"['Kakoos#NA1', 'Derek#VNM1', 'kevinnguyensocal...",1,0,0,0,0
4291,Adam,GB2,2,50,123,87,9,11,16547.05,16492.40,...,4.95,5.80,0.443763,182.25,"['FR1DGE#USA', 'Bouj#NA1', 'Bungo BALLS#BOOM',...",1,0,0,0,0
6131,Adam,NA02,2,78,31,19,10,9,9998.15,21992.35,...,5.45,11.25,0.494792,31.40,"['SIEGE MACHINE#NA1', 'FufuSauceGraine#Afrik',...",0,0,0,1,0
1554,Asleep,THSMS,2,63,129,101,16,4,8977.10,12579.95,...,3.20,12.30,0.647321,33.40,"['Wizard101 BF#Zach', 'Mori#88888', 'Nimb#CAT'...",0,0,0,1,0
4251,Asleep,Pengu,2,754,38,30,11,9,12483.30,16710.30,...,4.40,12.50,0.649378,30.90,"['Akadian Pasha#NA1', '邪悪な#Dev', 'repiv#0000',...",0,0,0,1,0
1176,Asuna,5103,0,74,304,250,14,6,12597.00,16788.90,...,3.65,13.75,0.671518,32.30,"['12 Pouces Mou#NA1', 'lyteskin#lyte', 'Sharpe...",0,0,0,1,0
8034,Asuna,1Yuki,1,707,130,106,13,7,14609.45,21501.35,...,3.45,4.70,0.385417,156.40,"['NoPause#클로버', 'Super Big Ego#NA1', 'Celerity...",0,0,0,0,1
